# Playing with Boxes

In this notebook, we will perform the following tasks in **pytorch as well as tensorflow**:
1. Create a tensor representation of a box
2. Represent specific boxes using your tensor representation
3. Create a function which calculates the intersection of two boxes
3. Create a differentiable function to compute "soft" volumes of boxes
4. Train one box to contain another


This notebook is intended to be self-contained, but you may find it beneficial to consult Section 3.1 to 3.4 in [Representing Joint Hierarchies with Box Embeddings](https://openreview.net/pdf?id=J246NSqR_l).

## 1. Create a box parameterization

A "box" is a product of intervals in $\mathbb R^n$, i.e.

$$X = \prod_{i=1}^d [x_{m,i}, x_{M,i}], \quad \text{where} \quad x_{M,i} \ge x_{m,i} \quad \text{ for all } \quad i.$$

However you would like, create a way of storing parameters which represent a box. Crucially, your parameterization should conform to the requirement above, namely any setting of your parameterization should represent a box that has positive side-lengths in each dimension, and you should be able to represent any box in $\mathbb R^n$. Furthermore, you should be able to return the min and max coordinates (i.e. the $x_{m,i}, x_{M, i}$ above) for your box. 

**Hint**: It might be benificial to create a "wrapper" class that holds the tensor(s) corresponding to the parameters of the box.

In [1]:
# Box parameterization
import numpy as np
import torch

class Box():
    def __init__(self, min_coords, max_coords, dtype=torch.float, requires_grad=False):
        if len(min_coords) != len(max_coords):
            raise ValueError("Dimensions do not match")
        minT = torch.tensor(min_coords, dtype=dtype, requires_grad=requires_grad)
        maxT = torch.tensor(max_coords, dtype=dtype, requires_grad=requires_grad)
        if not (minT <= maxT).all():
            raise ValueError("Maximum coordinates must be greater than minimum coordinates")
        
        self.n_dim = len(min_coords)
        self.min_coords = minT
        self.max_coords = maxT
        
    @classmethod
    def from_side_and_center(cls, side_length, center_coords, dtype=torch.float, requires_grad=False):
        if isinstance(side_length, int):
            side_length = [side_length] * len(center_coords)
        elif len(side_length) != len(center_coords):
            raise ValueError("Dimensions do not match")
        
        side_length = np.array(side_length)
        min_coords = center_coords - side_length/2
        max_coords = center_coords + side_length/2
        return cls(min_coords, max_coords, dtype)

## 2. Represent Specific Boxes
Using your chosen parameterization, represent a box `x` in $\mathbb R^{100}$ which has side-lengths $2$ centered at $(-1,\ldots, -1)$. 
Create another box `y` with min-coordinate $(1, \frac 1 2, \frac 1 3, \ldots, \frac 1 {100})$, and max-coordinate $(3 - \frac 1 {100}, 3 - \frac 1 {99}, \ldots, 2)$.


In [2]:
# Represent a specific box
x_side = 2
x_center = [-1]*100
x = Box.from_side_and_center(x_side, x_center)

y_min = [1/n for n in range(1, 101)]
y_max = [3 - k for k in reversed(y_min)]
y = Box(y_min, y_max, requires_grad=True)

# 3. Create an Intersection Function

The box intersection operation takes two boxes as input and returns their intersection. In one-dimension, this is as follows:

$$\text{Int}([x_m, x_M], [y_m, y_M]) = (\max(x_m, y_m), \min(x_M, y_M)).$$

Generalize this to $n$-dimensions, and write an intersection function which operates on your box parameterization.

Take the intersection of the two boxes `x` and `y` you created in step 2, and display the min and max coordinates of the intersection.

In [3]:
# Intersection Function
def box_intersection(boxA, boxB):
    intersection_min = torch.max(boxA.min_coords, boxB.min_coords).detach().numpy()
    intersection_max = torch.min(boxA.max_coords, boxB.max_coords).detach().numpy()
    return (intersection_min, intersection_max)
    
box_intersection(x, y)

(array([1.        , 0.5       , 0.33333334, 0.25      , 0.2       ,
        0.16666667, 0.14285715, 0.125     , 0.11111111, 0.1       ,
        0.09090909, 0.08333334, 0.07692308, 0.07142857, 0.06666667,
        0.0625    , 0.05882353, 0.05555556, 0.05263158, 0.05      ,
        0.04761905, 0.04545455, 0.04347826, 0.04166667, 0.04      ,
        0.03846154, 0.03703704, 0.03571429, 0.03448276, 0.03333334,
        0.03225806, 0.03125   , 0.03030303, 0.02941176, 0.02857143,
        0.02777778, 0.02702703, 0.02631579, 0.02564103, 0.025     ,
        0.02439024, 0.02380952, 0.02325581, 0.02272727, 0.02222222,
        0.02173913, 0.0212766 , 0.02083333, 0.02040816, 0.02      ,
        0.01960784, 0.01923077, 0.01886792, 0.01851852, 0.01818182,
        0.01785714, 0.01754386, 0.01724138, 0.01694915, 0.01666667,
        0.01639344, 0.01612903, 0.01587302, 0.015625  , 0.01538462,
        0.01515152, 0.01492537, 0.01470588, 0.01449275, 0.01428571,
        0.01408451, 0.01388889, 0.01369863, 0.01

## 4. Create a "Soft" Volume Function
The "softplus volume" of a box is defined as

$$\text{SoftVol}[X] = \prod_{i=1}^d \log(1 + \exp(x_{M,i} - x_{m,i})).$$

Create this volume function, and attempt to use it to determine which of the boxes `x` and `y` from part (2) are larger. What issues do you encounter? Can you mitigate these issues?

It's better to use logarithmic scale since product over dimensions might lead to numerical instability.

In [4]:
# Soft Volume Function
def soft_volume(max_coords, min_coords):
    lse = torch.log(1 + torch.exp(max_coords - min_coords))
    return torch.sum(torch.log(lse))

volX = soft_volume(x.max_coords, x.min_coords)
volY = soft_volume(y.max_coords, y.min_coords)
print(volX.item())
print(volY.item())
if volX > volY:
    print("Box X has larger volume than Box Y")
elif volX == volY:
    print("Both boxes, X and Y, has the same volume")
else:
    print("Box Y has larger volume than Box X")

75.46786499023438
108.06694793701172
Box Y has larger volume than Box X


## 5. Training Boxes
We say box $X$ contains box $Y$ if, for each dimension $i$, we have
$$x_{m,i} < y_{m,i} < y_{M,i} < x_{M,i}.$$

Freezing the coordinates of box `x` from earlier, use gradient-descent to train box `y` such that it is contained in `x`.

**Hint:** Visualize typical boxes in 2-dimensions, and note that if a box $Y$ were contained in $X$ then $\text{Vol}(\text{Int}(X, Y)) = \text{Vol}(Y)$.

**Hint:** You can use/invent any loss function as long as it has a local minima that corresponds to satisfying the inequality given above.

In [5]:
# Training boxes
alpha = 10
beta = 10

max_c = y.max_coords.detach().clone().requires_grad_(True)
min_c = y.min_coords.detach().clone().requires_grad_(True)

max_c
learning_rate = 5e-4
for t in range(1000):
    loss = torch.sum(alpha*torch.max(x.min_coords - min_c, torch.tensor(0, dtype=torch.float))) + \
            torch.sum(alpha*torch.max(max_c - x.max_coords, torch.tensor(0, dtype=torch.float))) + \
            torch.sum(beta*torch.max(min_c - max_c, torch.tensor(0, dtype=torch.float)))
    if t%50 == 0:
        print(t, loss.item())
    loss.backward()
    with torch.no_grad():
        max_c -= learning_rate * max_c.grad
        min_c -= learning_rate * min_c.grad

        max_c.grad = None
        min_c.grad = None

0 2948.12646484375
50 2698.12060546875
100 2448.114990234375
150 2198.109130859375
200 1948.104736328125
250 1698.104736328125
300 1448.1051025390625
350 1198.1053466796875
400 948.2056884765625
450 701.906005859375
500 455.7585754394531
550 214.6334228515625
600 19.79534149169922
650 5.2833452224731445
700 2.450007200241089
750 1.2000083923339844
800 0.0
850 0.0
900 0.0
950 0.0


In [6]:
y_ = Box(min_c.detach().numpy(), max_c.detach().numpy())

In [7]:
box_ins = box_intersection(x, y_)

In [8]:
assert (box_ins[0] == y_.min_coords.detach().numpy()).all()
assert (box_ins[1] == y_.max_coords.detach().numpy()).all()